In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import json

In [2]:
base_url="https://www.transfermarkt.co.uk"
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [45]:
#takes years in STR (2020,2019, etc)
#takes leauge accorinng to TransferMarkt format
 #English Premier = GB1
 #Spain La Liga = ES1
 #Italt Serie A = IT1
 #Germany BundesLiga = L1
 #France Ligue1 = FR1

        
def get_data(league, leagueID, years): #Returns a single DF with TransferMarkt ID, name, position, team, and dict of injuries 
    base_url="https://www.transfermarkt.co.uk"
    headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    #Collect the list of all teams and generate links from the teams list
#     page = 'https://www.transfermarkt.co.uk/premier-league/startseite/wettbewerb/'+league+'/plus/?saison_id='+str(years)
    page = 'https://www.transfermarkt.co.uk/'+league+'/startseite/wettbewerb/'+leagueID+'/plus/?saison_id='+str(years)
    tree = requests.get(page, headers = headers)
    soup = BeautifulSoup(tree.content, 'html.parser')

    teams = soup.find("table", class_="items").find("tbody")
    link=teams.find_all("a", class_="vereinprofil_tooltip")
    links=link[::3]
    teamlist=[]
    linklist=[]
    
    for i in range(len(links)):
        teamlist.append(re.search("\/(.*?)\/",links[i]["href"])[1])
        linklist.append(base_url+links[i]["href"])

    output=pd.DataFrame() # Create empty df 
    output2=pd.DataFrame()
    
    for j in range(len(linklist)): # Append data of each team to the output DF
        soup1=BeautifulSoup(requests.get(linklist[j], headers = headers).content, 'html.parser')
        playertable=soup1.find("table", class_="items") #Player list
        playertable_inj=soup1.findAll("a", {"class": "spielprofil_tooltip"})[::2] #Player list (sepeartely for injury)
        #Injury list comes from another table on a separate URL
        ids=[]
        injury_list=[]
        
        for player in set(playertable_inj):
            link_maker=str(player).split(" ")[2].split("/")
            name=link_maker[1]
            id1=link_maker[4][:-1]
            ids.append(id1)
            link_made="https://www.transfermarkt.co.uk/"+name+"/verletzungen/spieler/"+id1
            soup_inj=BeautifulSoup(requests.get(link_made, headers = headers).content, 'html.parser')
            
            inja = soup_inj.find("table", class_="items") #Check if there is injury data for the player
            list_1=[]
            if inja is not None:
                injuries=inja.find("tbody")
                duration=[int(re.findall("\d+", x.text)[0]) for x in injuries.find_all("td",class_="rechts")[::2]]
                inj_type=[x.text for x in injuries.find_all("td",class_="hauptlink")[::2]]
                seasons=[x.text for x in injuries.find_all("td",class_="zentriert")[::3]]
                began=[x.text for x in injuries.find_all("td",class_="zentriert")[1::3]]
                end=[x.text for x in injuries.find_all("td",class_="zentriert")[2::3]]
                details = ["injury_type", "season", "duration", "began", "end"]
                
                for n in range(len(inj_type)):
                    list_1.extend([seasons[n],inj_type[n],duration[n],began[n],end[n]])
            
            injury_list.append(list_1)


        rows = playertable.find_all('td',class_="posrela")
        players=[x.find(class_="spielprofil_tooltip").text for x in rows]
        positions=[x.find_all("td")[-1].text for x in rows]
        team=[teamlist[j]]*len(players)
        data=pd.DataFrame(list(zip(ids,players,positions,team)))
        data2=pd.DataFrame(injury_list)
        output=output.append(data)
        output2=output2.append(data2)

    return_pd=pd.concat([output,output2],axis=1)
    
    return return_pd




In [46]:
league_ids = ['GB1', 'ES1', 'IT1', 'L1', 'FR1']
leagues = ['premier-league', 'laliga', 'serie-a', 'bundesliga', 'ligue-1',]

In [47]:
prem = get_data(leagues[0], league_ids[0], 2020)
esp = get_data(leagues[1], league_ids[1], 2020)
italy = get_data(leagues[2], league_ids[2], 2020)
ger = get_data(leagues[3], league_ids[3], 2020)
fr = get_data(leagues[4], league_ids[4], 2020)

In [48]:
allLeagueDFs = [prem, esp, italy, ger, fra]
allLeagueData = pd.concat(allLeagueDFs)

In [1]:
allLeagueData

NameError: name 'allLeagueData' is not defined

In [50]:
def set_col_names(return_pd):
    
    return_pd2 = return_pd
    col_names = [None] * len(return_pd.columns)
    first_four = ['id', 'name', 'position', 'team']
    iterative_col_names = ['season', 'injury', 'days_of_injury', 'start_date', 'end_date'] * (len(return_pd.columns)-4)
    for i in range(4):
        col_names[i] = first_four[i]
    for j in range(4,len(return_pd.columns)):
        col_names[j] = iterative_col_names[j-4]
    
    return_pd2.columns = col_names
    
    
    return return_pd2

In [53]:
leaguedata = set_col_names(allLeagueData)
leaguedata.to_csv('allLeagueData.csv')